In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from urllib.parse import urlparse

OFFICIAL_BASE = "https://housing.seoul.go.kr"

# =====================================================
# 1️⃣ URL 정규화
# =====================================================
def canonicalize_url(url):
    if not isinstance(url, str):
        return ""
    url = url.strip()

    if url.startswith("/"):
        return OFFICIAL_BASE + url

    parsed = urlparse(url)
    if parsed.path.startswith("/site/main/"):
        return OFFICIAL_BASE + parsed.path

    return url


# =====================================================
# 2️⃣ 필드 매핑
# =====================================================
FIELD_MAP = {
    "지원대상": "eligibility",
    "신청자격": "eligibility",
    "대상": "eligibility",

    "지원내용": "benefit",
    "사업내용": "benefit",
    "지원항목": "benefit",
    "지원사항": "benefit",
    "지원범위": "benefit",

    "신청방법": "apply_process",
    "지원절차": "apply_process",
    "접수방법": "apply_process",
    "제출서류": "apply_process",
    "구비서류": "apply_process",
    "신청서류": "apply_process",
    "절차": "apply_process",

    "신청기간": "apply_period",
    "접수기간": "apply_period",
    "모집기간": "apply_period",
    "공고기간": "apply_period",
}

def get_standard_key(label):
    return FIELD_MAP.get(label.strip(), None)


# =====================================================
# 3️⃣ table 기반 추출
# =====================================================
def extract_from_table(soup):
    result = {}
    for table in soup.select("table"):
        for tr in table.select("tr"):
            cells = [c.get_text(" ", strip=True) for c in tr.select("th, td")]
            if len(cells) >= 2:
                key = get_standard_key(cells[0])
                if key:
                    result[key] = " ".join(cells[1:])
    return result


# =====================================================
# 4️⃣ dl 기반 추출
# =====================================================
def extract_from_dl(soup):
    result = {}
    for dt in soup.select("dt"):
        dd = dt.find_next_sibling("dd")
        if not dd:
            continue
        key = get_standard_key(dt.get_text(strip=True))
        if key:
            result[key] = dd.get_text(" ", strip=True)
    return result


# =====================================================
# 5️⃣ 텍스트 fallback
# =====================================================
def extract_from_text(soup):
    result = {}
    full_text = soup.get_text("\n", strip=True)

    labels = list(FIELD_MAP.keys())

    for label in labels:
        std_key = get_standard_key(label)
        if std_key in result:
            continue

        pattern = re.compile(
            rf"{label}\s*\n(.*?)(?=\n(?:{'|'.join(labels)})\s*\n|\Z)",
            re.DOTALL
        )
        match = pattern.search(full_text)
        if match:
            value = match.group(1).strip()
            if len(value) > 3000:
                value = value[:3000]
            result[std_key] = value

    return result


# =====================================================
# 6️⃣ 상세 페이지 파싱
# =====================================================
def parse_policy_page(url):

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": OFFICIAL_BASE,
    }

    resp = requests.get(url, headers=headers, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    full_text = soup.get_text("\n", strip=True)

    data = {}
    data.update(extract_from_table(soup))
    data.update(extract_from_dl(soup))

    fallback = extract_from_text(soup)
    for k, v in fallback.items():
        data.setdefault(k, v)

    return {
        "eligibility": data.get("eligibility", ""),
        "benefit": data.get("benefit", ""),
        "apply_process": data.get("apply_process", ""),
        "apply_period": data.get("apply_period", ""),
        "full_text": full_text[:5000]
    }


# =====================================================
# 7️⃣ 실행 (기존 대상 + 요약 유지)
# =====================================================
def crawl_and_parse(input_csv, output_csv):

    df = pd.read_csv(input_csv, encoding="utf-8-sig")

    results = []

    print(f"총 {len(df)}개 링크 분석 시작")

    for i, row in df.iterrows():

        url = canonicalize_url(row["링크"])

        print(f"[{i+1}/{len(df)}] {row['정책명'][:20]}")

        new_row = {
            "대상": row.get("대상", ""),
            "정책명": row.get("정책명", ""),
            "요약": row.get("요약", ""),
            "링크": url
        }

        try:
            parsed = parse_policy_page(url)
            new_row.update(parsed)
        except Exception as e:
            new_row["error"] = str(e)

        results.append(new_row)
        time.sleep(random.uniform(0.5, 1.0))

    final_df = pd.DataFrame(results)
    final_df.to_csv(output_csv, index=False, encoding="utf-8-sig")

    print("\n완료.")
    print(final_df.head())


if __name__ == "__main__":
    crawl_and_parse(
        "seoul_housing_policies_final.csv",
        "seoul_housing_structured.csv"
    )


총 89개 링크 분석 시작
[1/89] 신혼부부 임차보증금 반환보증 보증료 
[2/89] 전세보증금 반환보증 보증료 지원
[3/89] 신혼부부 임차보증금 이자지원
[4/89] 건물 에너지 효율화 사업 융자지원
[5/89] 주택개량 신축융자지원
[6/89] 안심 집수리 보조사업
[7/89] 집수리 아카데미
[8/89] 슬레이트 처리 및 지붕 개량
[9/89] 공동체 주택
[10/89] 신축주택 매입임대주택
[11/89] 신혼희망타운
[12/89] 희망의 집수리 사업
[13/89] 주거급여 수급자 지원(임차급여)
[14/89] 서울형 주택바우처(특정바우처)
[15/89] 서울형 주택바우처(일반바우처)
[16/89] 주거급여 수급자 지원(수선유지급여)
[17/89] 육아협동조합주택
[18/89] 예술인 협동조합주택
[19/89] 1인창조기업인 원룸 주택
[20/89] 청년안심주택
[21/89] 보증금지원형 장기안심주택
[22/89] 기존주택 전세임대주택
[23/89] 1인가구 주택관리서비스
[24/89] 서울형 주택바우처 (반지하 거주가구 
[25/89] 도시형생활주택
[26/89] 장기전세 주택
[27/89] 행복주택
[28/89] 국민임대주택
[29/89] 공공임대주택
[30/89] 전세보증금 반환보증 보증료 지원
[31/89] 청년임차보증금 이자지원
[32/89] 건물 에너지 효율화 사업 융자지원
[33/89] 주택개량 신축융자지원
[34/89] 안심 집수리 보조사업
[35/89] 집수리 아카데미
[36/89] 슬레이트 처리 및 지붕 개량
[37/89] 공동체 주택
[38/89] 신축주택 매입임대주택
[39/89] 청년전세임대주택
[40/89] 청년월세지원
[41/89] 희망의 집수리 사업
[42/89] 보호종료아동 및 쉼터퇴소 청소년 대상
[43/89] 주거급여 수급자 지원(임차급여)
[44/89] 서울형 주택바우처(특정바우처)
[45/89] 서울형 주택바우처(일반바우처)
[46/89] 주거급여 수급자 지원(수선유지급여)
[47/89] 청년협동조합주택
[48